In [1]:
import reservoirpy as rpy
from reservoirpy.datasets import (lorenz, mackey_glass, narma)
from reservoirpy.observables import (rmse, rsquare, nrmse, mse)
import numpy as np
import math
import pandas as pd
from functools import partial
import sys
import os
import time
import matplotlib.pyplot as plt
from memory_profiler import memory_usage
import copy

# Add the parent directory to the sys.path list
sys.path.append(os.path.abspath('../'))
import NAS.NAS
from joblib import Parallel, delayed
import warnings
warnings.filterwarnings("ignore")

rpy.verbosity(0)
output_dim = 1

In [2]:
def nrmseGESN(yTrue, preds):
    norm = 0
    mse = 0
    targetMean = np.mean(yTrue)
    for i, target in enumerate(yTrue):
        norm+=(target - targetMean)**2
        mse+=(target - preds[i])**2
    return math.sqrt(mse/norm)

def nmse(y, o):
    assert len(o) == len(y), "Both arrays must have the same length."
    sigma2 = np.var(y)  # variance of y
    i = len(y)
    error = sum((o[t] - y[t])**2 for t in range(i))
    return error / (i * sigma2)

def mae(y, o):
    assert len(o) == len(y), "Both arrays must have the same length."
    i = len(y)
    error = sum(abs(o[t] - y[t]) for t in range(i))
    return error / i

def mape(y, o):
    assert len(o) == len(y), "Both arrays must have the same length."
    i = len(y)
    error = sum(abs(o[t] - y[t]) / abs(y[t]) for t in range(i))
    return (error / i) * 100

def nrmseDBESN(yTrue, preds):
    norm = 0
    mse = 0
    targetMean = np.mean(yTrue)
    for i, target in enumerate(yTrue):
        norm+=(target - targetMean)**2
        mse+=(target - preds[i])**2
    return math.sqrt(mse/norm)

def r_squared(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    numerator = np.sum((y_true - y_pred)**2)
    denominator = np.sum((y_true - np.mean(y_true))**2)
    
    return 1 - (numerator / denominator)

def smape(A, F):
    tmp = 2 * np.abs(F - A) / (np.abs(A) + np.abs(F))
    len_ = np.count_nonzero(~np.isnan(tmp))
    if len_ == 0 and np.nansum(tmp) == 0: # Deals with a special case
        return 100
    return 100 / len_ * np.nansum(tmp)

def nrmse(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    rmse = np.sqrt(np.mean((y_true - y_pred)**2))
    mean_norm = np.linalg.norm(np.mean(y_true))
    
    return rmse / mean_norm

In [6]:
import NAS.utils


def getData():
    water = pd.read_csv("../datasets/Water.csv").to_numpy()
    trainLen = math.floor(len(water)*0.5)
    valLen = math.floor(len(water)*0.7)
    
    train_in = water[0:trainLen, :18]
    train_out = water[0:trainLen, 18:]
    val_in = water[trainLen:valLen, :18]
    val_out = water[trainLen:valLen, 18:]
    test_in = water[valLen:, :18]
    test_out = water[valLen:, 18:]
    return train_in, train_out, val_in, val_out, test_in, test_out

trainX, trainY, valX, valY, testX, testY = getData()

architecture = {'nodes': [{'type': 'Input', 'params': {'input_dim': 18}}, {'type': 'Reservoir', 'params': {'units': 2027, 'lr': 0.3826527162122251, 'sr': 0.6616215786597966, 'mu': -0.07780949008139451, 'sigma': 0.8407792483010825, 'learning_rate': 0.007590124316023448, 'input_connectivity': 0.3683303695774643, 'rc_connectivity': 0.19221187614289564, 'fb_connectivity': 0.22974931380273494}}, {'type': 'Ridge', 'params': {'output_dim': 18, 'ridge': 4.195942443537313e-05}}], 'edges': [[0, 1], [1, 2]]}
results = NAS.utils.evaluateArchitecture(architecture, np.concatenate([trainX, valX]), np.concatenate([trainY, valY]), testX, testY, [mse])
print(results)

([0.0061899196746492465], 'Model-73': Model('Input-9', 'Reservoir-3', 'Ridge-17'))


In [35]:
architecture["nodes"][0]["params"]["input_dim"]

1

In [47]:
import NAS.memory_estimator
from scipy.special import comb

def getInputDimension(architecture, idx):
    if idx==0:
        return architecture["nodes"][0]["params"]["input_dim"]
    inputDims = 0
    for edge in architecture["edges"]:
        if edge[1]!=idx: continue
        inputDim = getInputDimension(architecture, edge[0])
        inputNode = architecture["nodes"][edge[0]]
        outputDim = 0
        if inputNode["type"]=="Input":
            outputDim = inputDim
        elif inputNode["type"]=="Reservoir" or inputNode["type"]=="IPReservoir":
            outputDim = inputNode["params"]["units"]
        elif inputNode["type"]=="NVAR":
            effective_linear_dim = inputNode["params"]["delay"] * inputDim
            nonlinear_dim = comb(effective_linear_dim + inputNode["params"]["order"] - 1, inputNode["params"]["order"], exact=True)
            outputDim = effective_linear_dim + nonlinear_dim
        elif inputNode["type"]=="Ridge" or inputNode["type"]=="RLS" or inputNode["type"]=="LMS":
            outputDim = inputNode["params"]["output_dim"]
        inputDims+=outputDim

    return inputDims

getInputDimension(architecture, 6)

2535

In [23]:
from reservoirpy.nodes import (Reservoir, IPReservoir, NVAR, RLS, Input)

numInputs = 20
inputDim = 4
output_dim = 2
input = np.random.random((numInputs, inputDim))
model = Reservoir(100)
output = model.run(input)

output.shape

(20, 100)

In [21]:
from scipy.special import comb

delay = 3
order = 2

effective_linear_dim = delay * input.shape[1]

# Calculate memory for output
nonlinear_dim = comb(effective_linear_dim + order - 1, order, exact=True)
total_output_dim = effective_linear_dim + nonlinear_dim

print(total_output_dim)

54


In [7]:
for i in range(10):
    model = NAS.NAS.constructModel({'nodes': [{'type': 'Input', 'params': {}}, {'type': 'Reservoir', 'params': {'units': 2056, 'sr': 0.495, 'lr': 0.44}}, {'type': 'Ridge', 'params': {'output_dim': 6, 'ridge': 8.3e-7}}, ], 'edges': [[0, 1], [1, 2]]})
    model = NAS.NAS.trainModel(model, trainX, trainY)
    prevOutput = valX[0]
    preds = []
    for _ in range(len(valX)):
        pred = NAS.NAS.runModel(model, prevOutput)
        prevOutput = pred
        preds.append(pred[0])
    preds = np.array(preds)
    print("Performance", nrmse(valY, preds), r_squared(valY, preds))

Performance 0.22104962103346504 0.9999998336661783
Performance 0.23838567202711847 0.9999998065533048
Performance 0.2612360040215153 0.9999997676904409


KeyboardInterrupt: 

In [5]:
def getData():
    data = np.load('../data/Neutral_normed_2801.npy')
    from scipy import stats
    data = stats.zscore(data)
    data.shape

    trainLen = 2000
    valLen = 500
    testLen = 500
    train_in = data[0:trainLen]
    train_out = data[0+1:trainLen+1]
    val_in = data[trainLen:trainLen+valLen]
    val_out = data[trainLen+1:trainLen+valLen+1]
    test_in = data[trainLen+valLen:trainLen+valLen+testLen]
    test_out = data[trainLen+valLen+1:trainLen+valLen+testLen+1]
    return train_in, train_out, val_in, val_out, test_in, test_out
    
trainX, trainY, valX, valY, testX, testY = getData()

def nrmse(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    rmse = np.sqrt(np.mean((y_true - y_pred)**2))
    mean_norm = np.linalg.norm(np.mean(y_true))
    
    return rmse / mean_norm

architecture = {'nodes': [{'type': 'Input', 'params': {}}, {'type': 'Reservoir', 'params': {'units': 667, 'lr': 0.9500549414677324, 'sr': 1.4167120000597517, 'input_connectivity': 0.36350119807586495, 'rc_connectivity': 0.1511429334846635, 'fb_connectivity': 0.2091759221008611}}, {'type': 'Reservoir', 'params': {'units': 2484, 'lr': 0.40415873498029464, 'sr': 1.7446303891236545, 'input_connectivity': 0.10068601266694016, 'rc_connectivity': 0.38642618631525544, 'fb_connectivity': 0.49323769973643483}}, {'type': 'Ridge', 'params': {'output_dim': 6, 'ridge': 2.52961697706393e-05}}], 'edges': [[0, 1], [0, 2], [1, 3], [2, 3]]}
model = NAS.NAS.constructModel(architecture)
model = NAS.NAS.trainModel(model, trainX, trainY)
preds = NAS.NAS.runModel(model, valX)
prevOutput = valX[0]
preds = []
for _ in range(len(valX)):
    pred = NAS.NAS.runModel(model, prevOutput)
    prevOutput = pred
    preds.append(pred[0])
preds = np.array(preds)
print(nrmse(valY, preds))


# plt.plot(preds)
# plt.plot(valY)
# plt.show()

587.4470335653682


In [11]:
model = NAS.NAS.constructModel(architecture)
# nrmse, r_square = NAS.NAS.evaluateModelAutoRegressive2(model, trainX, trainY, valX, valY)

def nrmse(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    rmse = np.sqrt(np.mean((y_true - y_pred)**2))
    mean_norm = np.linalg.norm(np.mean(y_true))
    
    return rmse / mean_norm
    
def r_squared(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    numerator = np.sum((y_true - y_pred)**2)
    denominator = np.sum((y_true - np.mean(y_true))**2)
    return 1 - (numerator / denominator)

model = NAS.NAS.trainModel(model, trainX, trainY)
prevOutput = valX[0]
preds = []
for _ in range(len(valX)):
    pred = NAS.NAS.runModel(model, prevOutput)
    prevOutput = pred
    preds.append(pred[0])
preds = np.array(preds)
print(nrmse(valY, preds))


0.04661032952789324


In [4]:
goodArchitecture = {'nodes': [{'type': 'Input', 'params': {}}, {'type': 'NVAR', 'params': {'delay': 4, 'order': 2, 'strides': 1}}, {'type': 'Reservoir', 'params': {'units': 649, 'lr': 0.5975855910585006, 'sr': 1.289221609815766, 'input_connectivity': 0.40010488298206637, 'rc_connectivity': 0.4090786446967834, 'fb_connectivity': 0.09404210228238652}}, {'type': 'NVAR', 'params': {'delay': 4, 'order': 2, 'strides': 2}}, {'type': 'IPReservoir', 'params': {'units': 234, 'lr': 0.9203822465220324, 'sr': 0.5177516030001565, 'mu': 0.0661244738966123, 'sigma': 1.4035618806583001, 'learning_rate': 0.00327840184422856, 'input_connectivity': 0.14644358488436113, 'rc_connectivity': 0.3713680820702342, 'fb_connectivity': 0.273826121341471}}, {'type': 'NVAR', 'params': {'delay': 4, 'order': 2, 'strides': 2}}, {'type': 'Ridge', 'params': {'output_dim': 1, 'ridge': 6.045376465443952e-05}}], 'edges': [[0, 1], [0, 2], [0, 3], [1, 4], [3, 5], [2, 6], [4, 6], [5, 6]]}

model = NAS.NAS.constructModel(goodArchitecture)
model = NAS.NAS.trainModel(model, np.concatenate([trainX, valX[:-20]]), np.concatenate([trainY, valY[:-20]]))
prevOutput = valX[-20]
preds = []
for j in range(20+len(testX)):
    pred = NAS.NAS.runModel(model, prevOutput)
    prevOutput = pred
    preds.append(pred[0])

print(nrmse(testY, preds[-len(testY):]), r_squared(testY, preds[-len(testY):]))

0.03571277158613166 0.9897931229697442


In [9]:
# https://www.sciencedirect.com/science/article/pii/S0925231222014291
# Parameterizing echo state networks for multi-step time series prediction
# Neutral normed dataset

def getData():
    data = np.load('../data/Neutral_normed_2801.npy')

    trainLen = 1901
    valLen = 399
    testLen = 500
    train_in = data[0:trainLen]
    train_out = data[0+1:trainLen+1]
    val_in = data[trainLen:trainLen+valLen]
    val_out = data[trainLen+1:trainLen+valLen+1]
    test_in = data[trainLen+valLen:trainLen+valLen+testLen]
    test_out = data[trainLen+valLen+1:trainLen+valLen+testLen+1]
    return train_in, train_out, val_in, val_out, test_in, test_out

trainX, trainY, valX, valY, testX, testY = getData()

gaParams = {
    "evaluator": partial(evaluate, trainX=trainX, trainY=trainY, valX=valX, valY=valY),
    "generator": partial(NAS.generateRandomArchitecture, sampleX=trainX[:100], sampleY=trainY[:100]),
    "populationSize": 15,
    "eliteSize": 2,
    "stagnationReset": 5,
    "generations": 22,
    "minimizeFitness": True,
    "logModels": True,
    "seedModels": [],
    "crossoverProbability": 0.7,
    "mutationProbability": 0.2,
    "earlyStop": 0
}

nrmseErrors = []
r2Errors = []
for i in range(1):
    models, performances, architectures = NAS.runGA(gaParams)
    model = models[0]
    startInput = testX[0]
    prevOutput = testX[0]
    preds = []
    for j in range(len(testX)):
        pred = NAS.runModel(model, prevOutput)
        prevOutput = pred
        preds.append(pred[0])
    preds = np.array(preds)
    performance = r_squared(testY, preds)
    print("Performance", performance, nrmse(testY, preds))
    nrmseErrors.append(nrmse(testY, preds))
    r2Errors.append(performance)

print(np.array(nrmseErrors).mean(), np.array(nrmseErrors).std())
print(np.array(r2Errors).mean(), np.array(r2Errors).std())

Performance 0.999958227868178 7.311027348144948
Performance 0.9995362054529383 22.118367018388145
Performance -1.2067514774276298 2696.8280088904735
Performance -7.385636140212117 2739.3385181107597
Performance 0.9999629568174395 6.1284839531925925
Performance 0.9804485629026151 137.29728648428116
Performance 0.9999843941124462 4.420965116672592
Performance 0.9999883553787924 4.301596708526212
Performance 0.9999590021977057 6.849569508367799
Performance 0.9853479619681731 119.301864090258
574.3895687229065


In [6]:
# https://www.sciencedirect.com/science/article/pii/S0925231222014291
# Parameterizing echo state networks for multi-step time series prediction
# Lorenz dataset

def getData():
    data = np.load('../data/Lorenz_normed_2801.npy')

    trainLen = 1957
    valLen = 399
    testLen = 444
    train_in = data[0:trainLen]
    train_out = data[0+1:trainLen+1]
    val_in = data[trainLen:trainLen+valLen]
    val_out = data[trainLen+1:trainLen+valLen+1]
    test_in = data[trainLen+valLen:trainLen+valLen+testLen]
    test_out = data[trainLen+valLen+1:trainLen+valLen+testLen+1]
    return train_in, train_out, val_in, val_out, test_in, test_out

trainX, trainY, valX, valY, testX, testY = getData()

gaParams = {
    "evaluator": partial(evaluate, trainX=trainX, trainY=trainY, valX=valX, valY=valY),
    "generator": partial(NAS.generateRandomArchitecture, sampleX=trainX[:100], sampleY=trainY[:100]),
    "populationSize": 15,
    "eliteSize": 2,
    "stagnationReset": 5,
    "generations": 22,
    "minimizeFitness": True,
    "logModels": True,
    "seedModels": [],
    "crossoverProbability": 0.7,
    "mutationProbability": 0.2,
    "earlyStop": 0
}

nrmseErrors = []
r2Errors = []
for i in range(1):
    models, performances, architectures = NAS.runGA(gaParams)
    model = models[0]
    startInput = testX[0]
    prevOutput = testX[0]
    preds = []
    for j in range(len(testX)):
        pred = NAS.runModel(model, prevOutput)
        prevOutput = pred
        preds.append(pred[0])
    preds = np.array(preds)
    performance = r_squared(testY, preds)
    print("Performance", performance, nrmse(testY, preds))
    nrmseErrors.append(nrmse(testY, preds))
    r2Errors.append(performance)

print(np.array(nrmseErrors).mean(), np.array(nrmseErrors).std())
print(np.array(r2Errors).mean(), np.array(r2Errors).std())


ValueError: Ridge-10is  expecting data of shape (6,) but received shape (3,).

In [ ]:
def getLorenzData():
    data = np.array(lorenz(n_timesteps=10000, rho=28, sigma=10, beta=8/3, x0=[1, 1, 0]))
    train = data[:4000]
    val = data[4000:8000]
    test = data[8000:]
    trainX = train[:-1]
    trainY = train[1:]
    valX = val[:-1]
    valY = val[1:]
    testX = test[:-1]
    testY = test[1:]
    return trainX, trainY, valX, valY, testX, testY

def getMackeyData():
    data = np.array(mackey_glass(n_timesteps=10000, ))
    train = data[:4000]
    val = data[4000:8000]
    test = data[8000:]
    trainX = train[:-1]
    trainY = train[1:]
    valX = val[:-1]
    valY = val[1:]
    testX = test[:-1]
    testY = test[1:]
    return trainX, trainY, valX, valY, testX, testY

In [3]:
def getData():
    sunspots = pd.read_csv("../datasets/Sunspots.csv")
    data = sunspots.loc[:,"Monthly Mean Total Sunspot Number"]
    data1 = np.expand_dims(data[:-20], axis=1)
    data2 = np.expand_dims(data[10:-10], axis=1)
    data3 = np.expand_dims(data[20:], axis=1)
    y = data3[1:]
    x = np.concatenate([data1, data2, data3], axis=-1)[:-1]

    trainX = x[:1600]
    trainY = y[:1600]
    valX = x[1600:2100]
    valY = y[1600:2100]
    testX = x[2100:3174]
    testY = y[2100:3174]
    return trainX, trainY, valX, valY, testX, testY

trainX, trainY, valX, valY, testX, testY = getData()

gaParams = {
    "evaluator": partial(evaluate, trainX=trainX, trainY=trainY, valX=valX, valY=valY),
    "generator": partial(NAS.generateRandomArchitecture, sampleX=trainX[:100], sampleY=trainY[:100]),
    "populationSize": 15,
    "eliteSize": 3,
    "stagnationReset": 5,
    "generations": 5,
    "minimizeFitness": True,
    "logModels": False,
    "seedModels": [],
    "crossoverProbability": 0.7,
    "mutationProbability": 0.2,
    "earlyStop": 0
}

errors = []
for i in range(20):
    models, performances, architectures = NAS.runGA(gaParams)
    model = models[0]
    preds = NAS.runModel(model, testX)
    performance = nrmseGESN(testY, preds)
    print("Performance", performance)
    errors.append(performance)

print(np.array(errors).mean(), np.array(errors).std())

In [5]:
def getData():
    data = np.array(narma(n_timesteps=10000, order=10, a1=0.3, a2=0.05, b=1.5, c=0.1))
    train = data[:4000]
    val = data[4000:8000]
    test = data[8000:]
    trainX = train[:-1]
    trainY = train[1:]
    valX = val[:-1]
    valY = val[1:]
    testX = test[:-1]
    testY = test[1:]
    return trainX, trainY, valX, valY, testX, testY
trainX, trainY, valX, valY, testX, testY = getData()

gaParams = {
    "evaluator": partial(NAS.NAS.evaluateArchitecture, trainX=trainX, trainY=trainY, valX=valX, valY=valY),
    "generator": partial(NAS.NAS.generateRandomArchitecture, sampleX=trainX[:2], sampleY=trainY[:2]),
    "populationSize": 15,
    "eliteSize": 1,
    "stagnationReset": 5,
    "generations": 10,
    "minimizeFitness": True,
    "logModels": True,
    "seedModels": [],
    "crossoverProbability": 0.7,
    "mutationProbability": 0.2,
    "earlyStop": 0,
    "n_jobs": 5
}

nmseErrors = []
maeErrors = []
mapeErrors = []
for i in range(1):
    models, performances, architectures = NAS.NAS.runGA(gaParams)
    model = models[0]
    preds = NAS.NAS.runModel(model, testX)
    
    performance = nmse(testY, preds)
    print("Performance", nmse(testY, preds), mae(testY, preds), mape(testY, preds))
    nmseErrors.append(nmse(testY, preds))
    maeErrors.append(mae(testY, preds))
    mapeErrors.append(mape(testY, preds))

print(np.array(nmseErrors).mean(), np.array(nmseErrors).std())
print(np.array(maeErrors).mean(), np.array(maeErrors).std())
print(np.array(mapeErrors).mean(), np.array(mapeErrors).std())

In [6]:
def getData():
    sunspots = pd.read_csv("../datasets/Sunspots.csv")
    data = sunspots.loc[:,"Monthly Mean Total Sunspot Number"]
    data = np.expand_dims(data, axis=1)
    train = data[:2000]
    val = data[2000:2500]
    test = data[2500:]
    trainX = train[:-1]
    trainY = train[1:]
    valX = val[:-1]
    valY = val[1:]
    testX = test[:-1]
    testY = test[1:]
    return trainX, trainY, valX, valY, testX, testY
trainX, trainY, valX, valY, testX, testY = getData()

gaParams = {
    "evaluator": partial(NAS.NAS.evaluateArchitecture, trainX=trainX, trainY=trainY, valX=valX, valY=valY),
    "generator": partial(NAS.NAS.generateRandomArchitecture, sampleX=trainX[:2], sampleY=trainY[:2]),
    "populationSize": 15,
    "eliteSize": 1,
    "stagnationReset": 5,
    "generations": 10,
    "minimizeFitness": True,
    "logModels": True,
    "seedModels": [],
    "crossoverProbability": 0.7,
    "mutationProbability": 0.2,
    "earlyStop": 0,
    "n_jobs": 5
}

nmseErrors = []
maeErrors = []
mapeErrors = []
for i in range(1):
    models, performances, architectures = NAS.NAS.runGA(gaParams)
    model = models[0]
    preds = NAS.NAS.runModel(model, testX)
    
    performance = nmse(testY, preds)
    print("Performance", nmse(testY, preds), mae(testY, preds), mape(testY, preds))
    nmseErrors.append(nmse(testY, preds))
    maeErrors.append(mae(testY, preds))
    mapeErrors.append(mape(testY, preds))

print(np.array(nmseErrors).mean(), np.array(nmseErrors).std())
print(np.array(maeErrors).mean(), np.array(maeErrors).std())
print(np.array(mapeErrors).mean(), np.array(mapeErrors).std())

inf {'nodes': [{'type': 'Input', 'params': {}}, {'type': 'Reservoir', 'params': {'units': 2096, 'lr': 0.7129354139625268, 'sr': 0.6038712987483915, 'input_connectivity': 0.12379645556203929, 'rc_connectivity': 0.15107131768020632, 'fb_connectivity': 0.4564053934069279}}, {'type': 'LMS', 'params': {'output_dim': 1, 'alpha': 0.79379162946974}}, {'type': 'Ridge', 'params': {'output_dim': 1, 'ridge': 2.6839333416104074e-05}}, {'type': 'RLS', 'params': {'output_dim': 1, 'alpha': 0.838800877796766}}, {'type': 'Ridge', 'params': {'output_dim': 1, 'ridge': 2.9986109681454498e-05}}, {'type': 'RLS', 'params': {'output_dim': 1, 'alpha': 0.2449566053752632}}], 'edges': [[0, 1], [0, 2], [0, 3], [1, 4], [3, 5], [2, 6], [4, 6], [5, 6]]}
inf {'nodes': [{'type': 'Input', 'params': {}}, {'type': 'Ridge', 'params': {'output_dim': 1, 'ridge': 1.6715727733560162e-05}}, {'type': 'LMS', 'params': {'output_dim': 1, 'alpha': 0.32778680193485976}}, {'type': 'LMS', 'params': {'output_dim': 1, 'alpha': 0.16912171